# Build and test the normalized difference container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/norm-diff/

## Goal

Create a container and run the normalized difference step in the container image.


## Setup the environment

In [ ]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [ ]:
cat ${WORKSPACE}/water-bodies/command-line-tools/norm_diff/Dockerfile

Build the container using `podman`:

In [ ]:
podman build --format docker -t localhost/norm-diff:latest ${WORKSPACE}/water-bodies/command-line-tools/norm_diff


Show the `norm-diff` help:

In [ ]:
podman run --rm -it --env=PYTHONPATH=/app localhost/norm-diff:latest python -m app --help

## Test the normalized difference step in the container

Generate the normalized difference between the green and nir bands:

In [ ]:
podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/crop_green.tif,target=/inputs/crop_green.tif,readonly \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/crop_nir.tif,target=/inputs/crop_nir.tif,readonly \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/norm-diff:latest \
    python \
    -m \
    app \
    /inputs/crop_green.tif \
    /inputs/crop_nir.tif

List the outputs:

In [ ]:
tree ${RUNTIME}